In [1]:
import os

import pandas as pd
import numpy as np
import plotly.express as px

from scipy.stats import pearsonr

# Table of Contents

1. [Data loading](#data-loading)
2. [General analysis](#general-analysis)
    - [System sizes](#system-sizes)
    - [System investments](#system-investments)
3. [Comparison](#comparison)
    - [Total savings](#total-savings)
    - [Capacity costs savings](#capacity-costs-savings)
4. [Correlations](#correlations)
    - [On absolute savings](#on-absolute-savings)
    - [On relative savings](#on-relative-savings)

# Data loading

In [2]:
URI = os.getenv("DB_URI")

In [3]:
master = pd.read_sql("SELECT * FROM vea_industrial_load_profiles.master", URI)
master.set_index("id", inplace=True)
master.sort_index(inplace=True)

# in the authors database, the following columns are already present in master
# this will lead to problems later on, so we drop them here
# if you have created your own features by the "create_master_features.py", you
# do not need to worry about it
cols_to_drop = ["energy_costs_eur", "capacity_costs_eur", "total_costs_eur"]
for col_to_drop in cols_to_drop:
    try:
        master.drop(columns=col_to_drop, inplace=True)
    except KeyError as e:
        continue

master.head()

,sector_group_id,sector_group,capacity_price_over_2500h_eur_per_kw,energy_price_over_2500h_eur_per_kwh,capacity_price_under_2500h_eur_per_kw,energy_price_under_2500h_eur_per_kwh,zip_code,grid_level,number_of_peak_load,peak_load_kw,mean_load_kw,variance_kw,total_energy_kwh,full_load_hours_h,is_over_2500h,std_kw
id,,,,,,,,,,,,,,,,
0,B,Mining and quarrying,94.60,0.0060,14.67,0.0379,85,MS,1,2227.36,646.151302,423413.418622,5675793.04,2548.215394,True,650.702250
1,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,91,MS,1,1096.00,208.694900,109792.561757,1833176.00,1672.605839,False,331.349607
2,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,92,MS,1,816.00,129.444604,36662.220510,1137041.40,1393.433088,False,191.473812
3,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,85,MS,1,701.60,114.201389,19954.089684,1003145.00,1429.796180,False,141.258946
4,C,Manufacturing industry,101.04,0.0078,12.60,0.0432,30,MS,2,3480.00,1845.483607,610080.465591,16210728.00,4658.255172,True,781.076479


In [4]:
baseline = pd.read_sql("SELECT * FROM vea_results.overview WHERE name LIKE '%%base%%'", URI)
baseline["id"] = baseline["name"].str.split("_").str[0].astype(int)
baseline.set_index("id", inplace=True)
baseline.sort_index(inplace=True)
baseline.head()

,name,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,,
0,0_baseline,1.021643e+06,271558.090967,41249.388603,2227.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.334450e+06,0.0,0.0
1,1_baseline,3.299717e+05,12033.847188,132976.873344,1096.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.749824e+05,0.0,0.0
2,2_baseline,2.046675e+05,7464.085529,99004.679424,816.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.111362e+05,0.0,0.0
3,3_baseline,1.805661e+05,6585.125290,85124.611622,701.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.722758e+05,0.0,0.0
4,4_baseline,2.917931e+06,884063.074775,55353.715200,3480.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.857348e+06,0.0,0.0


In [5]:
storage = pd.read_sql("SELECT * FROM vea_results.overview WHERE name LIKE '%%storage_only'", URI)
storage["id"] = storage["name"].str.split("_").str[0].astype(int)
storage.set_index("id", inplace=True)
storage.sort_index(inplace=True)
storage.head()

,name,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,,
0,0_storage_only,1.018851e+06,270816.128970,41249.388603,2227.360000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.330917e+06,0.000000,0.000000
1,1_storage_only,3.293101e+05,12009.720415,112571.591043,927.818956,72705.633117,5658.350233,255.107485,30272.587940,2355.978453,168.181044,0.0,0.0,0.0,4.619058e+05,8014.328685,102978.221057
2,2_storage_only,2.043740e+05,7453.384611,77439.892170,638.262276,76183.293100,5929.000763,267.309800,31992.790344,2489.854017,177.737724,0.0,0.0,0.0,2.976862e+05,8418.854780,108176.083444
3,3_storage_only,1.800912e+05,6567.806727,75835.912954,625.042224,27228.142451,2119.043045,95.537342,13780.399758,1072.466119,76.557776,0.0,0.0,0.0,2.656864e+05,3191.509164,41008.542208
4,4_storage_only,2.909959e+06,881647.601893,55353.715200,3480.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,3.846960e+06,0.000000,0.000000


# General analysis

In [6]:
total_profiles_analyzed = len(storage)
profiles_using_storage = storage[storage["inverter_invest_eur"] > 0]
n_profiles_using_storage = len(profiles_using_storage)

print(f"{total_profiles_analyzed=}")
print(f"{n_profiles_using_storage=}")
print(f"Percentage of profiles using storage: {((n_profiles_using_storage / total_profiles_analyzed) * 100):.2f} %")

total_profiles_analyzed=5353
n_profiles_using_storage=2853
Percentage of profiles using storage: 53.30 %


## System sizes

In [7]:
storage_size = profiles_using_storage["storage_capacity_kwh"].copy()
storage_size.name = "Storage capacity"
print(storage_size.describe().drop("count").to_markdown())

print("")
inverter_size = profiles_using_storage["inverter_capacity_kw"].copy()
inverter_size.name = "Inverter capacity"
print(inverter_size.describe().drop("count").to_markdown())


fig_df = profiles_using_storage.copy()
fig_df = fig_df.rename(columns={
    "storage_capacity_kwh": "Storage",
    "inverter_capacity_kw": "Inverter"})
fig = px.box(
    data_frame=fig_df,
    x=["Inverter", "Storage"],
    title="Battery system sizes")
fig.update_layout(xaxis_title="Capacity in kWh (storage) / kW (inverter)", yaxis_title="")
fig.update_xaxes(range=[0, 200])
fig.show()

|      |   Storage capacity |
|:-----|-------------------:|
| mean |        80.3042     |
| std  |       283.607      |
| min  |         0.00157895 |
| 25%  |         4.96316    |
| 50%  |        19.7937     |
| 75%  |        63.5575     |
| max  |      8539.07       |

|      |   Inverter capacity |
|:-----|--------------------:|
| mean |             72.3453 |
| std  |            498.168  |
| min  |              0.006  |
| 25%  |             10.2505 |
| 50%  |             27.7879 |
| 75%  |             61.6    |
| max  |          25586.3    |


## System investments

In [8]:
storage_invest = profiles_using_storage["storage_invest_eur"].copy()
storage_invest.name = "Absolute storage investment"
print(storage_invest.describe().drop("count").to_markdown())

print("")
inverter_invest = profiles_using_storage["inverter_invest_eur"].copy()
inverter_invest.name = "Absolute inverter investment"
print(inverter_invest.describe().drop("count").to_markdown())


fig_df = profiles_using_storage.copy()
fig_df = fig_df.rename(columns={
    "storage_invest_eur": "Storage",
    "inverter_invest_eur": "Inverter"})
fig = px.box(
    data_frame=fig_df,
    x=["Inverter", "Storage"],
    title="Battery system investments")
fig.update_layout(xaxis_title="Storage system investments in €", yaxis_title="")
fig.update_xaxes(range=[0, 50000])
fig.show()

|      |   Absolute storage investment |
|:-----|------------------------------:|
| mean |               22886.7         |
| std  |               80828.1         |
| min  |                   0.45        |
| 25%  |                1414.5         |
| 50%  |                5641.2         |
| 75%  |               18113.9         |
| max  |                   2.43364e+06 |

|      |   Absolute inverter investment |
|:-----|-------------------------------:|
| mean |                13022.2         |
| std  |                89670.2         |
| min  |                    1.08        |
| 25%  |                 1845.09        |
| 50%  |                 5001.82        |
| 75%  |                11088           |
| max  |                    4.60554e+06 |


# Comparison

In [9]:
abs_diff = baseline.drop(columns="name") - storage.drop(columns="name")

# drop those that could not be optimized
abs_diff.dropna(subset="total_yearly_costs_eur", inplace=True)

abs_diff.head()

,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,
0,2791.373626,741.961997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,3533.335623,0.000000,0.000000
1,661.563319,24.126773,20405.282301,168.181044,-72705.633117,-5658.350233,-255.107485,-30272.587940,-2355.978453,-168.181044,0.0,0.0,0.0,13076.643708,-8014.328685,-102978.221057
2,293.422377,10.700918,21564.787254,177.737724,-76183.293100,-5929.000763,-267.309800,-31992.790344,-2489.854017,-177.737724,0.0,0.0,0.0,13450.055769,-8418.854780,-108176.083444
3,474.880156,17.318563,9288.698668,76.557776,-27228.142451,-2119.043045,-95.537342,-13780.399758,-1072.466119,-76.557776,0.0,0.0,0.0,6589.388223,-3191.509164,-41008.542208
4,7972.489180,2415.472882,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,10387.962062,0.000000,0.000000


In [10]:
rel_diff = (baseline.drop(columns="name") - storage.drop(columns="name")) / baseline.drop(columns="name")

# drop those that could not be optimized
rel_diff.dropna(subset="total_yearly_costs_eur", inplace=True)

rel_diff

,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,
0,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002648,NaN,NaN
1,0.002005,0.002005,0.153450,0.153450,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,0.027531,-inf,-inf
2,0.001434,0.001434,0.217816,0.217816,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,0.043229,-inf,-inf
3,0.002630,0.002630,0.109119,0.109119,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,0.024201,-inf,-inf
4,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002693,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5354,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002704,NaN,NaN
5355,0.002730,0.002730,0.037830,0.037830,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,0.003182,-inf,-inf
5356,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002710,NaN,NaN


In [11]:
# merge savings onto master (with features)
abs_diff_with_master = pd.merge(left=abs_diff, right=master, how="left", left_index=True, right_index=True)
abs_diff_with_master.head()

abs_diff_with_master["std_by_mean"] = abs_diff_with_master["std_kw"] / abs_diff_with_master["mean_load_kw"]
abs_diff_with_master["std_by_peak"] = abs_diff_with_master["std_kw"] / abs_diff_with_master["peak_load_kw"]
abs_diff_with_master["peak_by_mean"] = abs_diff_with_master["peak_load_kw"] / abs_diff_with_master["mean_load_kw"]

In [12]:
# merge savings onto master (with features)
rel_diff_with_master = pd.merge(left=rel_diff, right=master, how="left", left_index=True, right_index=True)
rel_diff_with_master.head()

rel_diff_with_master["std_by_mean"] = rel_diff_with_master["std_kw"] / rel_diff_with_master["mean_load_kw"]
rel_diff_with_master["std_by_peak"] = rel_diff_with_master["std_kw"] / rel_diff_with_master["peak_load_kw"]
rel_diff_with_master["peak_by_mean"] = rel_diff_with_master["peak_load_kw"] / rel_diff_with_master["mean_load_kw"]

## Total savings

In [13]:
abs_yearly_savings = abs_diff["total_yearly_costs_eur"].copy()
abs_yearly_savings.name = "Total yearly savings in eur"
print(abs_yearly_savings.describe().drop("count").to_markdown())

fig_df = abs_diff.copy()
fig_df = fig_df.rename(columns={"total_yearly_costs_eur": "Savings"})
fig = px.box(
    data_frame=fig_df,
    x=["Savings"],
    title="Total yearly savings")
fig.update_layout(xaxis_title="Total yearly savings in €", yaxis_title="")
fig.update_xaxes(range=[0, 10000])
fig.show()

|      |   Total yearly savings in eur |
|:-----|------------------------------:|
| mean |                      3613.75  |
| std  |                     10497.2   |
| min  |                         0     |
| 25%  |                       491.953 |
| 50%  |                      1378.72  |
| 75%  |                      3604.98  |
| max  |                    514178     |


In [14]:
rel_yearly_savings = 100 * rel_diff["total_yearly_costs_eur"].copy()
rel_yearly_savings.name = "Relative yearly savings in %"
print(rel_yearly_savings.describe().drop("count").to_markdown())
print("-----------------------")
n_profiles_1pct_yearly_savings = len(rel_diff[rel_diff["total_yearly_costs_eur"] > 0.01])
print(f"Number of profiles with more than 1% yearly savings: {n_profiles_1pct_yearly_savings}")
perc_profiles_1pct_yearly_savings = (n_profiles_1pct_yearly_savings / len(rel_diff)) * 100
print(f"Percentage of profiles with more than 1% yearly savings: {perc_profiles_1pct_yearly_savings:.2f} %")
n_profiles_2pct_yearly_savings = len(rel_diff[rel_diff["total_yearly_costs_eur"] > 0.02])
print(f"Number of profiles with more than 2% yearly savings: {n_profiles_2pct_yearly_savings}")
perc_profiles_2pct_yearly_savings = (n_profiles_2pct_yearly_savings / len(rel_diff)) * 100
print(f"Percentage of profiles with more than 2% yearly savings: {perc_profiles_2pct_yearly_savings:.2f} %")
n_profiles_3pct_yearly_savings = len(rel_diff[rel_diff["total_yearly_costs_eur"] > 0.03])
print(f"Number of profiles with more than 3% yearly savings: {n_profiles_3pct_yearly_savings}")
perc_profiles_3pct_yearly_savings = (n_profiles_3pct_yearly_savings / len(rel_diff)) * 100
print(f"Percentage of profiles with more than 3% yearly savings: {perc_profiles_3pct_yearly_savings:.2f} %")

fig_df = pd.DataFrame()
fig_df = rel_diff.copy() * 100
fig_df = fig_df.rename(columns={"total_yearly_costs_eur": "Savings"})
fig = px.box(
    data_frame=fig_df,
    x="Savings",
    title="Relative yearly savings")
fig.update_layout(xaxis_title="Relative yearly savings in %", yaxis_title="")
fig.update_xaxes(range=[0, 6])
fig.show()

|      |   Relative yearly savings in % |
|:-----|-------------------------------:|
| mean |                       1.52971  |
| std  |                       3.16932  |
| min  |                       0.244344 |
| 25%  |                       0.269174 |
| 50%  |                       0.271723 |
| 75%  |                       1.5311   |
| max  |                      75.0611   |
-----------------------
Number of profiles with more than 1% yearly savings: 1560
Percentage of profiles with more than 1% yearly savings: 29.15 %
Number of profiles with more than 2% yearly savings: 1152
Percentage of profiles with more than 2% yearly savings: 21.52 %
Number of profiles with more than 3% yearly savings: 835
Percentage of profiles with more than 3% yearly savings: 15.60 %


## Capacity costs savings

In [15]:
abs_cap_costs_yearly_savings = abs_diff["grid_capacity_costs_eur"].copy()
abs_cap_costs_yearly_savings.name = "Total yearly grid capacity costs savings in %"
print(abs_cap_costs_yearly_savings.describe().drop("count").to_markdown())

total_cap_saving_fig_df = abs_diff.copy()
total_cap_saving_fig_df = total_cap_saving_fig_df.rename(columns={"grid_capacity_costs_eur": "Savings"})
total_cap_savings_fig = px.box(
    data_frame=total_cap_saving_fig_df,
    x="Savings",
    title="Yearly capacity costs savings")
total_cap_savings_fig.update_layout(xaxis_title="Savings in €", yaxis_title="")
total_cap_savings_fig.update_xaxes(range=[0, 10e3])
total_cap_savings_fig.show()

|      |   Total yearly grid capacity costs savings in % |
|:-----|------------------------------------------------:|
| mean |                                  3242.98        |
| std  |                                 18307.3         |
| min  |                                     0           |
| 25%  |                                     0           |
| 50%  |                                    73.5222      |
| 75%  |                                  2106.71        |
| max  |                                     1.06138e+06 |


In [16]:
rel_cap_costs_yearly_savings = 100 * rel_diff["grid_capacity_costs_eur"].copy()
rel_cap_costs_yearly_savings.name = "Relative yearly grid capacity costs savings in %"
print(rel_cap_costs_yearly_savings.describe().drop("count").to_markdown())

rel_cap_savings_fig_df = pd.DataFrame()
rel_cap_savings_fig_df = rel_diff.copy() * 100
rel_cap_savings_fig_df.rename(columns={"grid_capacity_costs_eur": "Savings"}, inplace=True)
rel_cap_savings_fig = px.box(
    data_frame=rel_cap_savings_fig_df,
    x="Savings",
    title="Relative yearly capacity cost savings")
rel_cap_savings_fig.update_layout(xaxis_title="Savings in %", yaxis_title="")
rel_cap_savings_fig.update_xaxes(range=[0, 50])
rel_cap_savings_fig.show()

|      |   Relative yearly grid capacity costs savings in % |
|:-----|---------------------------------------------------:|
| mean |                                            9.20731 |
| std  |                                           14.0178  |
| min  |                                            0       |
| 25%  |                                            0       |
| 50%  |                                            1.16087 |
| 75%  |                                           14.9169  |
| max  |                                           99.9732  |


## Savings distributions

In [17]:
savings_hist_fig_df = rel_diff_with_master.copy() * 100
savings_hist_fig_df["is_over_2500h"] = savings_hist_fig_df["is_over_2500h"].astype(bool)

savings_hist = px.histogram(
    data_frame=savings_hist_fig_df,
    x="total_yearly_costs_eur",
    color="is_over_2500h",
    barmode="group",
    title="Distribution of relative yearly savings")
savings_hist.update_xaxes(range=[0, 10], title="Savings in %")

# Correlations

## On absolute savings

In [18]:
cols_to_drop = [
    "grid_level",
    "zip_code",
    "sector_group_id",
    "sector_group",
    "solar_invest_eur",
    "solar_annuity_eur",
    "solar_capacity_kwp"]
abs_correlations_df = abs_diff_with_master.drop(columns=cols_to_drop).corr()
px.imshow(abs_correlations_df, title="Correlation coefficients for total yearly savings")

In [19]:
fig_df = abs_correlations_df[["total_yearly_costs_eur"]].round(2)
fig_df.sort_values("total_yearly_costs_eur", inplace=True, ascending=False)
fig = px.bar(
    data_frame=fig_df,
    y="total_yearly_costs_eur",
    text_auto=True,
    title="Correlation between different load profile characteristics and total yearly savings")
fig.update_layout(yaxis_title="Correlation coefficient", xaxis_title="Variable")

In [20]:
df = pd.DataFrame()
i = 0
for var in abs_correlations_df.index:#[correlations_df["total_yearly_costs_eur"] > 0.3].index:
    corr, p_value = pearsonr(y=abs_diff_with_master.dropna()["total_yearly_costs_eur"], x=abs_diff_with_master.dropna()[var])
    df.loc[i, "var"] = var
    df.loc[i, "corr"] = corr
    df.loc[i, "p_value"] = p_value
    i += 1

df.sort_values("corr", ascending=False, ignore_index=True)

,var,corr,p_value
0,total_yearly_costs_eur,1.000000,0.000000e+00
1,grid_capacity_costs_eur,0.870315,0.000000e+00
2,grid_capacity_kw,0.854124,0.000000e+00
3,peak_load_kw,0.580533,0.000000e+00
4,std_kw,0.489817,4.102527e-310
5,mean_load_kw,0.446129,2.717942e-251
6,total_energy_kwh,0.446129,2.717942e-251
7,energy_costs_eur,0.365821,2.075243e-163
8,grid_energy_costs_eur,0.347917,5.858155e-147
9,variance_kw,0.319883,2.793086e-123


## On relative savings

In [21]:
cols_to_drop = [
    "grid_level",
    "zip_code",
    "sector_group_id",
    "sector_group",
    "solar_invest_eur",
    "solar_annuity_eur",
    "solar_capacity_kwp"]
rel_correlations_df = rel_diff_with_master.drop(columns=cols_to_drop).corr()
px.imshow(rel_correlations_df, title="Correlation coefficients for relative yearly savings")

In [22]:
fig_df = rel_correlations_df[["total_yearly_costs_eur"]].round(2)
fig_df.sort_values("total_yearly_costs_eur", inplace=True, ascending=False)
fig_df.dropna(inplace=True)
fig = px.bar(
    data_frame=fig_df,
    y="total_yearly_costs_eur",
    text_auto=True,
    title="Correlation between different load profile characteristics and relative yearly savings")
fig.update_layout(yaxis_title="Correlation coefficient", xaxis_title="Variable")

In [23]:
df = pd.DataFrame()
i = 0
for var in rel_correlations_df.index:
    if np.inf in rel_diff_with_master[var]:
        continue
    elif -np.inf in rel_diff_with_master[var]:
        continue
    elif np.nan in rel_diff_with_master[var]:
        continue
    elif rel_diff_with_master[var].isin([np.nan, np.inf, -np.inf]).any():
        continue
    if "storage" in var or "inverter" in var:
        continue

    corr, p_value = pearsonr(y=rel_diff_with_master["total_yearly_costs_eur"], x=rel_diff_with_master[var])
    df.loc[i, "var"] = var
    df.loc[i, "corr"] = corr
    df.loc[i, "p_value"] = p_value
    i += 1

df.sort_values("corr", ascending=False, ignore_index=True)

,var,corr,p_value
0,total_yearly_costs_eur,1.000000,0.000000e+00
1,grid_capacity_kw,0.813182,0.000000e+00
2,grid_capacity_costs_eur,0.813182,0.000000e+00
3,std_by_mean,0.487954,2.678775e-318
4,peak_by_mean,0.377786,3.632898e-181
5,energy_price_under_2500h_eur_per_kwh,0.214763,7.014716e-57
6,capacity_price_over_2500h_eur_per_kw,0.192926,4.836621e-46
7,capacity_price_under_2500h_eur_per_kw,0.083816,8.192652e-10
8,energy_price_over_2500h_eur_per_kwh,0.072840,9.575004e-08
9,number_of_peak_load,-0.021791,1.109388e-01


## Scatterplots

In [24]:
cap_vs_savings_fig_df = rel_diff_with_master.copy()
cap_vs_savings_fig_df.loc[cap_vs_savings_fig_df["is_over_2500h"], "capacity_price_eur"] = cap_vs_savings_fig_df.loc[cap_vs_savings_fig_df["is_over_2500h"], "capacity_price_over_2500h_eur_per_kw"].copy()
cap_vs_savings_fig_df.loc[~cap_vs_savings_fig_df["is_over_2500h"], "capacity_price_eur"] = cap_vs_savings_fig_df.loc[~cap_vs_savings_fig_df["is_over_2500h"], "capacity_price_under_2500h_eur_per_kw"].copy()
cap_vs_savings_fig_df["total_yearly_costs_eur"] *= 100
cap_vs_savings_fig_df = cap_vs_savings_fig_df[cap_vs_savings_fig_df.index.isin(profiles_using_storage.index)]
# cap_vs_savings_fig_df = cap_vs_savings_fig_df[cap_vs_savings_fig_df["capacity_price_eur"] < 200]

cap_vs_savings_fig = px.scatter(
    data_frame=cap_vs_savings_fig_df,
    x="capacity_price_eur",
    y="total_yearly_costs_eur",
    title="Grid capacity prices vs relative yearly savings",
    color="is_over_2500h",
    trendline="ols",
    trendline_color_override="black")
cap_vs_savings_fig.update_xaxes(title="Grid capacity costs in eur / kW")
cap_vs_savings_fig.update_yaxes(title="Relative yearly savings in %")
cap_vs_savings_fig.update_layout(showlegend=False)
cap_vs_savings_fig.show()